In [1]:
from tensorflow import keras 
from keras.models import Model
import os, shutil
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras import models
from keras.layers import Dropout
from tensorflow.keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras 
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
import tensorflow as tf
img_size=256

# ISIC

BASIC FILE STARTS WITH ISIC 2017 2019 AND 2020 ALREADY 

In [121]:
original_dataset_dir = "C:/Users/Migue/Desktop/thesis_data"
df= pd.read_csv(original_dataset_dir+'/ISICmerge.csv', sep=',')
df = df.iloc[:, :9]
df.image = df.image.apply(lambda x: x + '.jpg')
#drop downsample

df = df.loc[~df['image'].str.contains('downsampled')]
#drop 2020 duplicatges in df
isic2020_dup= pd.read_csv(r"C:/Users/Migue/Desktop/thesis_data/ISIC 2020/ISIC_2020_Training_Duplicates.csv", sep=',')
#df = df[~df['image'].isin(isic2020_dup.image_name_2)]
dfimages=[]
for i in df.image:
    dfimages.append(i)
isic2020_dupls=[]
isic2020_dup.image_name_2 = isic2020_dup.image_name_2.apply(lambda x: x + '.jpg')

for i in isic2020_dup.image_name_2:
    isic2020_dupls.append(i)
df=df.loc[~df['image'].isin(isic2020_dupls)]
dfimages=[]
for i in df.image:
    dfimages.append(i)
    
#drop 2020duplicates in files
#for file_name in os.listdir("C:/Users/Migue/Desktop/thesis_data/centralized_images/"):
 #   if file_name not in dfimages:
        #os.remove("C:/Users/Migue/Desktop/thesis_data/centralized_images/"+file_name)
        #print("removing: ",file_name)

In [127]:
df = df.fillna(0)

In [123]:
df["Benign"]=(df.NV + df.BKL+df.DF)
df["Other Diseases"]=(df.BCC +df.AK+ df.SCC+df.VASC)
df=df.drop(["NV","BCC","AK","BKL","DF","VASC","SCC"],axis=1)

In [129]:
df

image  MEL  Benign  Other Diseases
0      ISIC_0000000.jpg    0     1.0             0.0
1      ISIC_0000001.jpg    0     1.0             0.0
2      ISIC_0000002.jpg    1     0.0             0.0
3      ISIC_0000003.jpg    0     1.0             0.0
4      ISIC_0000004.jpg    1     0.0             0.0
...                 ...  ...     ...             ...
60482  ISIC_9999134.jpg    0     1.0             0.0
60483  ISIC_9999320.jpg    0     1.0             0.0
60484  ISIC_9999515.jpg    0     1.0             0.0
60485  ISIC_9999666.jpg    0     1.0             0.0
60486  ISIC_9999806.jpg    0     1.0             0.0

[57988 rows x 4 columns]

In [128]:
df.loc[(df['MEL'] == 0) & (df['Other Diseases'] == 0), 'Benign'] = 1


image             0
MEL               0
Benign            0
Other Diseases    0
dtype: int64

In [130]:

df['Benign'] = df['Benign'].astype(int)
df['Other Diseases'] = df['Other Diseases'].astype(int)

In [131]:
df

image  MEL  Benign  Other Diseases
0      ISIC_0000000.jpg    0       1               0
1      ISIC_0000001.jpg    0       1               0
2      ISIC_0000002.jpg    1       0               0
3      ISIC_0000003.jpg    0       1               0
4      ISIC_0000004.jpg    1       0               0
...                 ...  ...     ...             ...
60482  ISIC_9999134.jpg    0       1               0
60483  ISIC_9999320.jpg    0       1               0
60484  ISIC_9999515.jpg    0       1               0
60485  ISIC_9999666.jpg    0       1               0
60486  ISIC_9999806.jpg    0       1               0

[57988 rows x 4 columns]

# 7 POINT SYSTEM IMPORT-melanoma- in df and data transferred

In [132]:
seven_df= pd.read_csv(r"C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/meta/meta.csv", sep=',')
seven_df=seven_df[["diagnosis", "derm"]]
seven_df=seven_df.rename(columns={"derm":"image"})
seven_df[['folder', 'image']] = seven_df['image'].str.split('/', expand=True)
seven_df=seven_df.drop("folder",axis=1)

seven_df.diagnosis.value_counts()

put diseases in right column

In [133]:
seven_df['MEL']='0'
seven_df.loc[seven_df['diagnosis'].str.contains('melanoma'), 'MEL'] = 1

In [134]:

seven_df['Benign']='0'
seven_df.loc[seven_df['diagnosis'].str.contains('nevus'), 'Benign'] = 1
seven_df.loc[seven_df['diagnosis'].str.contains('seborrheic keratosis'), 'Benign'] = 1
seven_df.loc[seven_df['diagnosis'].str.contains('miscellaneous'), 'Benign'] = 1
seven_df.loc[seven_df['diagnosis'].str.contains('dermatofibroma'), 'Benign'] = 1

In [135]:
seven_df['Other Diseases']='0'
seven_df.loc[seven_df['diagnosis'].str.contains('vascular lesion'), 'Other Diseases'] = 1
seven_df.loc[seven_df['diagnosis'].str.contains('basal cell carcinoma'), 'Other Diseases'] = 1
seven_df.loc[seven_df['diagnosis'].str.contains('lentigo'), 'Other Diseases'] = 1
seven_df.loc[seven_df['diagnosis'].str.contains('melanosis'), 'Other Diseases'] = 1

In [136]:
seven_df=seven_df.drop("diagnosis", axis=1)

In [137]:
df = pd.concat([df, seven_df])

In [138]:
df

image MEL Benign Other Diseases
0     ISIC_0000000.jpg   0      1              0
1     ISIC_0000001.jpg   0      1              0
2     ISIC_0000002.jpg   1      0              0
3     ISIC_0000003.jpg   0      1              0
4     ISIC_0000004.jpg   1      0              0
...                ...  ..    ...            ...
1006        Fhl058.jpg   0      0              1
1007        Fhl060.jpg   0      0              1
1008        Fil082.jpg   0      0              1
1009       newl016.jpg   0      0              1
1010        Adl389.jpg   1      0              0

[58999 rows x 4 columns]

SEND FILES TO CENTRALIZED FILE

In [121]:
src_path = "C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images"
dst_path = 'C:/Users/Migue/Desktop/thesis_data/centralized_images'
x=0
for subdir, dirs, files in os.walk(src_path):
    for dir in dirs:
        dir_path = os.path.join(subdir, dir)
        print(f"Found folder: {dir_path}")
        for file in os.listdir(dir_path):
            if file in seven_df['image'].values:
                print(f"- {file}")
                src_file = os.path.join(dir_path, file)
                dst_file = os.path.join(dst_path, file)
                print(f"Sending file {src_file} to {dst_file}")
                shutil.copyfile(src_file, dst_file)
                x+=1
print(x)

Found folder: C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\A1l
- Aal002.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\A1l\Aal002.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Aal002.jpg
- Aal002bis.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\A1l\Aal002bis.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Aal002bis.jpg
- Aal004.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\A1l\Aal004.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Aal004.jpg
- Aal006.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\A1l\Aal006.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Aal006.jpg
- Aal008.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\A1l\Aal008.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Aal008.jpg
- Aal010.jpg
Sending file C:/Use

- Acl223.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\A3l\Acl223.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Acl223.jpg
- Acl225.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\A3l\Acl225.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Acl225.jpg
- Acl227.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\A3l\Acl227.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Acl227.jpg
- Acl229.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\A3l\Acl229.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Acl229.jpg
- Acl231.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\A3l\Acl231.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Acl231.jpg
- Acl233.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\A3l\Acl233.jpg to C:/Users/Migue/

- Ael452.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\Ael\Ael452.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Ael452.jpg
- Ael455.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\Ael\Ael455.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Ael455.jpg
- Ael459.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\Ael\Ael459.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Ael459.jpg
- Ael465.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\Ael\Ael465.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Ael465.jpg
- Ael467.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\Ael\Ael467.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Ael467.jpg
- Ael469.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\Ael\Ael469.jpg to C:/Users/Migue/

- FBL066.JPG
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\FBL\FBL066.JPG to C:/Users/Migue/Desktop/thesis_data/centralized_images\FBL066.JPG
- Fbl068.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\FBL\Fbl068.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Fbl068.jpg
- Fbl072.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\FBL\Fbl072.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Fbl072.jpg
- Fbl074.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\FBL\Fbl074.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Fbl074.jpg
- Fbl076.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\FBL\Fbl076.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Fbl076.jpg
- Fbl080.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\FBL\Fbl080.jpg to C:/Users/Migue/

- Fel038.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\FEL\Fel038.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Fel038.jpg
- Fel052.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\FEL\Fel052.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Fel052.jpg
- Fel064.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\FEL\Fel064.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Fel064.jpg
- Fel068.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\FEL\Fel068.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Fel068.jpg
- Fel070.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\FEL\Fel070.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Fel070.jpg
- Fel072.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\FEL\Fel072.jpg to C:/Users/Migue/

...

- Ndl021.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\NDL\Ndl021.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Ndl021.jpg
- Ndl022.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\NDL\Ndl022.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Ndl022.jpg
- Ndl025.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\NDL\Ndl025.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Ndl025.jpg
- Ndl027.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\NDL\Ndl027.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Ndl027.jpg
- Ndl028.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\NDL\Ndl028.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Ndl028.jpg
- Ndl031.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\NDL\Ndl031.jpg to C:/Users/Migue/

- Nel100.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\NEL\Nel100.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Nel100.jpg
Found folder: C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\New
- newl002.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\New\newl002.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\newl002.jpg
- newl006.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\New\newl006.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\newl006.jpg
- newl008.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\New\newl008.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\newl008.jpg
- newl010.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\New\newl010.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\newl010.jpg
- newl012.jpg
Sending file C:

- Ngl029.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\NGL\Ngl029.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Ngl029.jpg
- Ngl031.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\NGL\Ngl031.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Ngl031.jpg
- Ngl033.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\NGL\Ngl033.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Ngl033.jpg
- Ngl037.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\NGL\Ngl037.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Ngl037.jpg
- Ngl039.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\NGL\Ngl039.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Ngl039.jpg
- Ngl041.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\NGL\Ngl041.jpg to C:/Users/Migue/

- Nil086.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\NIL\Nil086.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Nil086.jpg
- Nil088.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\NIL\Nil088.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Nil088.jpg
- Nil090.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\NIL\Nil090.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Nil090.jpg
- Nil093.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\NIL\Nil093.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Nil093.jpg
- Nil096.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\NIL\Nil096.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Nil096.jpg
- Nil100.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\NIL\Nil100.jpg to C:/Users/Migue/

- Nml077.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\NML\Nml077.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Nml077.jpg
- Nml079.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\NML\Nml079.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Nml079.jpg
- Nml081.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\NML\Nml081.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Nml081.jpg
- Nml085.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\NML\Nml085.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Nml085.jpg
- Nml087.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\NML\Nml087.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\Nml087.jpg
- Nml089.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/7-point-data/release_v0/images\NML\Nml089.jpg to C:/Users/Migue/

# PH2- melanoma- in df and data transferred


In [139]:
ph2_df= pd.read_csv(r"C:/Users/Migue/Desktop/thesis_data/PH2Dataset/ph2_class.csv", sep=',')
df = pd.concat([df, ph2_df])
melanoma_class_ph2=ph2_df["image"]

send file to centralized data checking if it is melanoma

In [123]:
ph2_df["image"]

0     IMD058
1     IMD061
2     IMD063
3     IMD064
4     IMD065
5     IMD080
6     IMD085
7     IMD088
8     IMD090
9     IMD091
10    IMD168
11    IMD211
12    IMD219
13    IMD240
14    IMD242
15    IMD284
16    IMD285
17    IMD348
18    IMD349
19    IMD403
20    IMD404
21    IMD405
22    IMD407
23    IMD408
24    IMD409
25    IMD410
26    IMD413
27    IMD417
28    IMD418
29    IMD419
30    IMD406
31    IMD411
32    IMD420
33    IMD421
34    IMD423
35    IMD424
36    IMD425
37    IMD426
38    IMD429
39    IMD435
Name: image, dtype: object

In [140]:
df

image MEL Benign Other Diseases
0   ISIC_0000000.jpg   0      1              0
1   ISIC_0000001.jpg   0      1              0
2   ISIC_0000002.jpg   1      0              0
3   ISIC_0000003.jpg   0      1              0
4   ISIC_0000004.jpg   1      0              0
..               ...  ..    ...            ...
35            IMD424   1    NaN            NaN
36            IMD425   1    NaN            NaN
37            IMD426   1    NaN            NaN
38            IMD429   1    NaN            NaN
39            IMD435   1    NaN            NaN

[59039 rows x 4 columns]

In [125]:
src_path = "C:/Users/Migue/Desktop/thesis_data/PH2Dataset/PH2 Dataset images"
dst_path = 'C:/Users/Migue/Desktop/thesis_data/centralized_images'

for subdir, dirs, files in os.walk(src_path):
    for dir in dirs:
        if "Dermoscopic" in dir:
            dir_path = os.path.join(subdir, dir)
            for file in os.listdir(dir_path):
                if file[:6] in ph2_df["image"].values:
                    print(f"Found Melanoma file in this path: {dir_path}")
                    print(f"- {file}")
                    src_file = os.path.join(dir_path, file)
                    dst_file = os.path.join(dst_path, file)
                    print(f"Sending file {src_file} to {dst_file}")
                    print()
                    shutil.copyfile(src_file, dst_file)

Found Melanoma file in this path: C:/Users/Migue/Desktop/thesis_data/PH2Dataset/PH2 Dataset images\IMD058\IMD058_Dermoscopic_Image
- IMD058.bmp
Sending file C:/Users/Migue/Desktop/thesis_data/PH2Dataset/PH2 Dataset images\IMD058\IMD058_Dermoscopic_Image\IMD058.bmp to C:/Users/Migue/Desktop/thesis_data/centralized_images\IMD058.bmp

Found Melanoma file in this path: C:/Users/Migue/Desktop/thesis_data/PH2Dataset/PH2 Dataset images\IMD061\IMD061_Dermoscopic_Image
- IMD061.bmp
Sending file C:/Users/Migue/Desktop/thesis_data/PH2Dataset/PH2 Dataset images\IMD061\IMD061_Dermoscopic_Image\IMD061.bmp to C:/Users/Migue/Desktop/thesis_data/centralized_images\IMD061.bmp

Found Melanoma file in this path: C:/Users/Migue/Desktop/thesis_data/PH2Dataset/PH2 Dataset images\IMD063\IMD063_Dermoscopic_Image
- IMD063.bmp
Sending file C:/Users/Migue/Desktop/thesis_data/PH2Dataset/PH2 Dataset images\IMD063\IMD063_Dermoscopic_Image\IMD063.bmp to C:/Users/Migue/Desktop/thesis_data/centralized_images\IMD063.bmp

Found Melanoma file in this path: C:/Users/Migue/Desktop/thesis_data/PH2Dataset/PH2 Dataset images\IMD419\IMD419_Dermoscopic_Image
- IMD419.bmp
Sending file C:/Users/Migue/Desktop/thesis_data/PH2Dataset/PH2 Dataset images\IMD419\IMD419_Dermoscopic_Image\IMD419.bmp to C:/Users/Migue/Desktop/thesis_data/centralized_images\IMD419.bmp

Found Melanoma file in this path: C:/Users/Migue/Desktop/thesis_data/PH2Dataset/PH2 Dataset images\IMD420\IMD420_Dermoscopic_Image
- IMD420.bmp
Sending file C:/Users/Migue/Desktop/thesis_data/PH2Dataset/PH2 Dataset images\IMD420\IMD420_Dermoscopic_Image\IMD420.bmp to C:/Users/Migue/Desktop/thesis_data/centralized_images\IMD420.bmp

Found Melanoma file in this path: C:/Users/Migue/Desktop/thesis_data/PH2Dataset/PH2 Dataset images\IMD421\IMD421_Dermoscopic_Image
- IMD421.bmp
Sending file C:/Users/Migue/Desktop/thesis_data/PH2Dataset/PH2 Dataset images\IMD421\IMD421_Dermoscopic_Image\IMD421.bmp to C:/Users/Migue/Desktop/thesis_data/centralized_images\IMD421.bmp

# skinl2 data- melanoma- in df and data transferred

In [141]:
src_path = "C:/Users/Migue/Desktop/thesis_data/SKINL/mela"
dst_path = 'C:/Users/Migue/Desktop/thesis_data/centralized_images'

for subdir, dirs, files in os.walk(src_path):
    print(files)
    df_skinl2=pd.DataFrame({"image": files, "MEL":1})


['0003_Dermatoscopic.png', '0022_1_Dermatoscopic.png', '0022_2_Dermatoscopic.png', '0023_Dermatoscopic.png', '0024_Dermatoscopic.png', '0027_1_Dermatoscopic.png', '0027_2_Dermatoscopic.png', '0027_3_Dermatoscopic.png', '0027_4_Dermatoscopic.png', '0027_5_Dermatoscopic.png', '0027_Dermatoscopic.png', '0031_Dermatoscopic.png', '0046_1_Dermatoscopic.png', '0046_2_Dermatoscopic.png', '0059_1_Dermatoscopic.png', '0059_2_Dermatoscopic.png', '0059_3_Dermatoscopic.png', '0060_Dermatoscopic.png', '0061_Dermatoscopic.png', '0065_Dermatoscopic.png', '0071_1_Dermatoscopic.png', '0071_2_Dermatoscopic.png', '0071_3_Dermatoscopic.png', '0071_4_Dermatoscopic.png', '0080_Dermatoscopic.png', '0101_Dermatoscopic.png', '0121_Dermatoscopic.png', '0124_1_Dermatoscopic.jpg', '0124_2_Dermatoscopic.jpg', '0132_Dermatoscopic.png', '0137_Dermatoscopic.png', '0138_Dermatoscopic.png', '0187_Dermatoscopic.png', '0195_Dermatoscopic.png', '0203_Dermatoscopic.png', '0207_Dermatoscopic.png', '0218_Dermatoscopic.png', '

In [142]:
df = pd.concat([df, df_skinl2])

In [128]:
for subdir, dirs, files in os.walk(src_path):
    for file in files:
        src_file = os.path.join(subdir, file)
        dst_file = os.path.join(dst_path, file)
        print(f"Sending file {src_file} to {dst_file}")
        print()
        shutil.copyfile(src_file, dst_file)

Sending file C:/Users/Migue/Desktop/thesis_data/SKINL/mela\0003_Dermatoscopic.png to C:/Users/Migue/Desktop/thesis_data/centralized_images\0003_Dermatoscopic.png

Sending file C:/Users/Migue/Desktop/thesis_data/SKINL/mela\0022_1_Dermatoscopic.png to C:/Users/Migue/Desktop/thesis_data/centralized_images\0022_1_Dermatoscopic.png

Sending file C:/Users/Migue/Desktop/thesis_data/SKINL/mela\0022_2_Dermatoscopic.png to C:/Users/Migue/Desktop/thesis_data/centralized_images\0022_2_Dermatoscopic.png

Sending file C:/Users/Migue/Desktop/thesis_data/SKINL/mela\0023_Dermatoscopic.png to C:/Users/Migue/Desktop/thesis_data/centralized_images\0023_Dermatoscopic.png

Sending file C:/Users/Migue/Desktop/thesis_data/SKINL/mela\0024_Dermatoscopic.png to C:/Users/Migue/Desktop/thesis_data/centralized_images\0024_Dermatoscopic.png

Sending file C:/Users/Migue/Desktop/thesis_data/SKINL/mela\0027_1_Dermatoscopic.png to C:/Users/Migue/Desktop/thesis_data/centralized_images\0027_1_Dermatoscopic.png

Sending fi

# dermnet 


In [143]:
src_path = "C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data"
dst_path = 'C:/Users/Migue/Desktop/thesis_data/dermnetrelevant'
images=[]
for subdir, dirs, files in os.walk(src_path):
    for dir in dirs:
        dir_path = os.path.join(subdir, dir)
        for file in os.listdir(dir_path):
            images.append(file)

In [144]:
dermnet_df=pd.DataFrame({"image":images, "MEL":0,"Benign":0,"Other Diseases":0})

In [145]:
dermnet_df=dermnet_df[~dermnet_df['image'].duplicated()]
dermnet_df=dermnet_df[~dermnet_df['image'].isna()]

In [146]:
dermnet_df

image  MEL  Benign  Other Diseases
0            07Acne081101.jpg    0       0               0
1       07AcnePittedScars.jpg    0       0               0
2      07AcnePittedScars1.jpg    0       0               0
3       07PerioralDermEye.jpg    0       0               0
4      07PerioralDermNose.jpg    0       0               0
...                       ...  ...     ...             ...
19554    warts-plantar-98.jpg    0       0               0
19555    warts-plantar-99.jpg    0       0               0
19556   warts-treatment-1.jpg    0       0               0
19557   warts-treatment-2.jpg    0       0               0
19558   warts-treatment-3.jpg    0       0               0

[18856 rows x 4 columns]

In [17]:
dermnet_df["Other Diseases"].value_counts()

0    17813
1     1043
Name: Other Diseases, dtype: int64

In [18]:
dermnet_df["Benign"].value_counts()

0    15462
1     3394
Name: Benign, dtype: int64

In [19]:
dermnet_df["MEL"].value_counts()

0    18668
1      188
Name: MEL, dtype: int64

In [147]:
for subdir, dirs, files in os.walk(src_path):
    if "Melanoma" in subdir:
        for file in files:
            if "melanoma" in file or "maligna" in file:
                dermnet_df.loc[dermnet_df['image'].str.contains('melanoma'), 'MEL'] = 1
                dermnet_df.loc[dermnet_df['image'].str.contains('maligna'), 'MEL'] = 1

In [148]:
dermnet_df.loc[dermnet_df['image'].str.contains('nevus'), 'Benign'] = 1
dermnet_df.loc[dermnet_df['image'].str.contains('nevi'), 'Benign'] = 1
dermnet_df.loc[dermnet_df['image'].str.contains('seborrheic keratosis'), 'Benign'] = 1
dermnet_df.loc[dermnet_df['image'].str.contains('miscellaneous'), 'Benign'] = 1
dermnet_df.loc[dermnet_df['image'].str.contains('acne'), 'Benign'] = 1
dermnet_df.loc[dermnet_df['image'].str.contains('rosacea'), 'Benign'] = 1
dermnet_df.loc[dermnet_df['image'].str.contains('atopic'), 'Benign'] = 1 # atopic dermatitus
dermnet_df.loc[dermnet_df['image'].str.contains('Atopic'), 'Benign'] = 1 
dermnet_df.loc[dermnet_df['image'].str.contains('benign'), 'Benign'] = 1 
dermnet_df.loc[dermnet_df['image'].str.contains('dermatitis'), 'Benign'] = 1 
dermnet_df.loc[dermnet_df['image'].str.contains('eczema'), 'Benign'] = 1 
dermnet_df.loc[dermnet_df['image'].str.contains('dermatofibroma'), 'Benign'] = 1

In [149]:
dermnet_df.loc[dermnet_df['image'].str.contains('carcinoma'), 'Other Diseases'] = 1 #BCC
dermnet_df.loc[dermnet_df['image'].str.contains('vascular lesion'), 'Other Diseases'] = 1
dermnet_df.loc[dermnet_df['image'].str.contains('actinic'), 'Other Diseases'] = 1 #10% develop into cancer actinic-keratosis and actinic-cheilitis


In [150]:
dermnet_df = dermnet_df[~(dermnet_df[["MEL","Benign","Other Diseases"]] == 0).all(axis=1)]

In [151]:
dermnet_df

image  MEL  Benign  Other Diseases
13            07rosacea010206OK.jpg    0       1               0
28     07rosaceaTreated091806OK.jpg    0       1               0
42       acne-closed-comedo-002.jpg    0       1               0
43       acne-closed-comedo-003.jpg    0       1               0
44         acne-closed-comedo-1.jpg    0       1               0
...                             ...  ...     ...             ...
18333       eczema-herpeticum-7.jpg    0       1               0
18334       eczema-herpeticum-8.jpg    0       1               0
18335       eczema-herpeticum-9.jpg    0       1               0
18336       eczema-vaccinatum-1.jpg    0       1               0
18337       eczema-vaccinatum-2.jpg    0       1               0

[4625 rows x 4 columns]

In [152]:
df

image MEL Benign Other Diseases
0          ISIC_0000000.jpg   0      1              0
1          ISIC_0000001.jpg   0      1              0
2          ISIC_0000002.jpg   1      0              0
3          ISIC_0000003.jpg   0      1              0
4          ISIC_0000004.jpg   1      0              0
..                      ...  ..    ...            ...
48  159_1_Dermatoscopic.png   1    NaN            NaN
49  159_2_Dermatoscopic.png   1    NaN            NaN
50  159_3_Dermatoscopic.png   1    NaN            NaN
51  159_4_Dermatoscopic.png   1    NaN            NaN
52  159_5_Dermatoscopic.png   1    NaN            NaN

[59092 rows x 4 columns]

In [153]:
df = pd.concat([df, dermnet_df])

In [143]:
src_path = "C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data"
dst_path = 'C:/Users/Migue/Desktop/thesis_data/centralized_images'
x=0
for subdir, dirs, files in os.walk(src_path):
    for dir in dirs:
            dir_path = os.path.join(subdir, dir)
            for file in os.listdir(dir_path):
                if file in dermnet_df["image"].values:
                    print(f"- {file}")
                    src_file = os.path.join(dir_path, file)
                    dst_file = os.path.join(dst_path, file)
                    print(f"Sending file {src_file} to {dst_file}")
                    print()
                    shutil.copyfile(src_file, dst_file)
                    x+=1
print(x)

- 07rosacea010206OK.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Acne and Rosacea Photos\07rosacea010206OK.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\07rosacea010206OK.jpg

- 07rosaceaTreated091806OK.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Acne and Rosacea Photos\07rosaceaTreated091806OK.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\07rosaceaTreated091806OK.jpg

- acne-closed-comedo-002.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Acne and Rosacea Photos\acne-closed-comedo-002.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\acne-closed-comedo-002.jpg

- acne-closed-comedo-003.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Acne and Rosacea Photos\acne-closed-comedo-003.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\acne-closed-comedo-003.jpg

- acne-closed-comedo-1.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/de

- acne-cystic-119.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Acne and Rosacea Photos\acne-cystic-119.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\acne-cystic-119.jpg

- acne-cystic-12.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Acne and Rosacea Photos\acne-cystic-12.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\acne-cystic-12.jpg

- acne-cystic-120.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Acne and Rosacea Photos\acne-cystic-120.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\acne-cystic-120.jpg

- acne-cystic-121.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Acne and Rosacea Photos\acne-cystic-121.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\acne-cystic-121.jpg

- acne-cystic-122.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Acne and Rosacea Photos\acne-cystic-122.jpg to C:/Users/Migue/Des

- acne-cystic-48.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Acne and Rosacea Photos\acne-cystic-48.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\acne-cystic-48.jpg

- acne-cystic-49.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Acne and Rosacea Photos\acne-cystic-49.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\acne-cystic-49.jpg

- acne-cystic-5.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Acne and Rosacea Photos\acne-cystic-5.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\acne-cystic-5.jpg

- acne-cystic-50.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Acne and Rosacea Photos\acne-cystic-50.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\acne-cystic-50.jpg

- acne-cystic-51.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Acne and Rosacea Photos\acne-cystic-51.jpg to C:/Users/Migue/Desktop/thesis_da

- acne-infantile-12.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Acne and Rosacea Photos\acne-infantile-12.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\acne-infantile-12.jpg

- acne-infantile-13.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Acne and Rosacea Photos\acne-infantile-13.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\acne-infantile-13.jpg

- acne-infantile-14.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Acne and Rosacea Photos\acne-infantile-14.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\acne-infantile-14.jpg

- acne-infantile-15.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Acne and Rosacea Photos\acne-infantile-15.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\acne-infantile-15.jpg

- acne-infantile-16.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Acne and Rosacea Photos\acne-infanti

- acne-open-comedo-60.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Acne and Rosacea Photos\acne-open-comedo-60.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\acne-open-comedo-60.jpg

- acne-open-comedo-61.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Acne and Rosacea Photos\acne-open-comedo-61.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\acne-open-comedo-61.jpg

- acne-open-comedo-62.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Acne and Rosacea Photos\acne-open-comedo-62.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\acne-open-comedo-62.jpg

- acne-open-comedo-63.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Acne and Rosacea Photos\acne-open-comedo-63.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\acne-open-comedo-63.jpg

- acne-open-comedo-64.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Acne and R

...

- epidermal-nevus-29.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Seborrheic Keratoses and other Benign Tumors\epidermal-nevus-29.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\epidermal-nevus-29.jpg

- epidermal-nevus-3.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Seborrheic Keratoses and other Benign Tumors\epidermal-nevus-3.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\epidermal-nevus-3.jpg

- epidermal-nevus-30.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Seborrheic Keratoses and other Benign Tumors\epidermal-nevus-30.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\epidermal-nevus-30.jpg

- epidermal-nevus-31.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Seborrheic Keratoses and other Benign Tumors\epidermal-nevus-31.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\epidermal-nevus-31.jpg

- epidermal-nevus-32.jpg
Sending fi

- epidermal-nevus-inflammatory-6.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Seborrheic Keratoses and other Benign Tumors\epidermal-nevus-inflammatory-6.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\epidermal-nevus-inflammatory-6.jpg

- melanoma-mimic-1.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Seborrheic Keratoses and other Benign Tumors\melanoma-mimic-1.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\melanoma-mimic-1.jpg

- melanoma-mimic-10.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Seborrheic Keratoses and other Benign Tumors\melanoma-mimic-10.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\melanoma-mimic-10.jpg

- melanoma-mimic-11.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Seborrheic Keratoses and other Benign Tumors\melanoma-mimic-11.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\melanoma-mimic-11.jpg

- melano

- nevus-sebaceous-30.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Seborrheic Keratoses and other Benign Tumors\nevus-sebaceous-30.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\nevus-sebaceous-30.jpg

- nevus-sebaceous-31.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Seborrheic Keratoses and other Benign Tumors\nevus-sebaceous-31.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\nevus-sebaceous-31.jpg

- nevus-sebaceous-32.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Seborrheic Keratoses and other Benign Tumors\nevus-sebaceous-32.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\nevus-sebaceous-32.jpg

- nevus-sebaceous-33.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Seborrheic Keratoses and other Benign Tumors\nevus-sebaceous-33.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\nevus-sebaceous-33.jpg

- nevus-sebaceous-34.jpg
Sending

- acrodermatitis-enteropathica-1.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Systemic Disease\acrodermatitis-enteropathica-1.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\acrodermatitis-enteropathica-1.jpg

- acrodermatitis-enteropathica-2.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Systemic Disease\acrodermatitis-enteropathica-2.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\acrodermatitis-enteropathica-2.jpg

- acrodermatitis-enteropathica-3.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Systemic Disease\acrodermatitis-enteropathica-3.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\acrodermatitis-enteropathica-3.jpg

- acrodermatitis-enteropathica-4.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Systemic Disease\acrodermatitis-enteropathica-4.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\acrodermatitis-enteropathica-4.jpg



- eczema-herpeticum-36.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Warts Molluscum and other Viral Infections\eczema-herpeticum-36.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\eczema-herpeticum-36.jpg

- eczema-herpeticum-37.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Warts Molluscum and other Viral Infections\eczema-herpeticum-37.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\eczema-herpeticum-37.jpg

- eczema-herpeticum-38.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Warts Molluscum and other Viral Infections\eczema-herpeticum-38.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\eczema-herpeticum-38.jpg

- eczema-herpeticum-39.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/dermnet-kaggle data\Warts Molluscum and other Viral Infections\eczema-herpeticum-39.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\eczema-herpeticum-39.jpg

- eczema-herpeti

# dermIS dataset


In [154]:
Benign = "C:/Users/Migue/Desktop/thesis_data/Skin Lesion DERMIS Dataset/Benign"
MEL = "C:/Users/Migue/Desktop/thesis_data/Skin Lesion DERMIS Dataset/Malignant"

In [155]:
_Benign=[]
_MEL=[]
for subdir, dirs, files in os.walk(Benign):
    for file in files:
        _Benign.append(file)
        print(f"- {file}")
        src_file = os.path.join(Benign, file)
        dst_file = os.path.join(dst_path, file)
        print(f"Sending file {src_file} to {dst_file}")
        print()
        #shutil.copyfile(src_file, dst_file)
        
for subdir, dirs, files in os.walk(MEL):
    for file in files:
        _MEL.append(file)
        print(f"- {file}")
        src_file = os.path.join(MEL, file)
        dst_file = os.path.join(dst_path, file)
        print(f"Sending file {src_file} to {dst_file}")
        print()
        #shutil.copyfile(src_file, dst_file)
        

- Dermis Benign  (1).jpg
Sending file C:/Users/Migue/Desktop/thesis_data/Skin Lesion DERMIS Dataset/Benign\Dermis Benign  (1).jpg to C:/Users/Migue/Desktop/thesis_data/dermnetrelevant\Dermis Benign  (1).jpg

- Dermis Benign  (10).jpg
Sending file C:/Users/Migue/Desktop/thesis_data/Skin Lesion DERMIS Dataset/Benign\Dermis Benign  (10).jpg to C:/Users/Migue/Desktop/thesis_data/dermnetrelevant\Dermis Benign  (10).jpg

- Dermis Benign  (100).jpg
Sending file C:/Users/Migue/Desktop/thesis_data/Skin Lesion DERMIS Dataset/Benign\Dermis Benign  (100).jpg to C:/Users/Migue/Desktop/thesis_data/dermnetrelevant\Dermis Benign  (100).jpg

- Dermis Benign  (101).jpg
Sending file C:/Users/Migue/Desktop/thesis_data/Skin Lesion DERMIS Dataset/Benign\Dermis Benign  (101).jpg to C:/Users/Migue/Desktop/thesis_data/dermnetrelevant\Dermis Benign  (101).jpg

- Dermis Benign  (102).jpg
Sending file C:/Users/Migue/Desktop/thesis_data/Skin Lesion DERMIS Dataset/Benign\Dermis Benign  (102).jpg to C:/Users/Migue/D

In [156]:
mel = pd.DataFrame({'image': _MEL,'MEL': 1})
benign = pd.DataFrame({'image': _Benign,'Benign': 1})

dermis=pd.concat([mel,benign])
dermis = dermis.fillna(0)
dermis.MEL=dermis.MEL.astype(int)
dermis.Benign=dermis.Benign.astype(int)

In [157]:
dermis

image  MEL  Benign
0      Dermis Malignant (1).jpg    1       0
1     Dermis Malignant (10).jpg    1       0
2    Dermis Malignant (100).jpg    1       0
3    Dermis Malignant (101).jpg    1       0
4    Dermis Malignant (102).jpg    1       0
..                          ...  ...     ...
495     Dermis Benign  (95).jpg    0       1
496     Dermis Benign  (96).jpg    0       1
497     Dermis Benign  (97).jpg    0       1
498     Dermis Benign  (98).jpg    0       1
499     Dermis Benign  (99).jpg    0       1

[1000 rows x 3 columns]

In [158]:
df=pd.concat([df,dermis])

# 17k dataset


In [1070]:
import urllib

In [1071]:
#url = 'https://raw.githubusercontent.com/mattgroh/fitzpatrick17k/main/fitzpatrick17k.csv'

# download the CSV file and save it as 'fitzpatrick17k.csv'
#urllib.request.urlretrieve(url, 'fitzpatrick17k.csv')

# load the CSV file into a Pandas DataFrame
#fitz = pd.read_csv('fitzpatrick17k.csv')


In [159]:
fitzdata= pd.read_csv(r"C:/Users/Migue/Desktop/thesis_data/finalfitz17k/fitzpatrick17k.csv")
fitzdata=fitzdata.rename(columns={"md5hash":"image","label":"diagnosis"})
fitzdata=fitzdata[["image", "diagnosis"]]
fitzdata.image = fitzdata.image.apply(lambda x: x + '.jpg')

In [160]:
fitzdata['MEL']='0'
fitzdata.loc[fitzdata['diagnosis'].str.contains('melanoma'), 'MEL'] = 1
fitzdata.loc[fitzdata['diagnosis'].str.contains('lentigo maligna'), 'MEL'] = 1

fitzdata['Other Diseases']='0'
fitzdata.loc[fitzdata['diagnosis'].str.contains('vascular lesion'), 'Other Diseases'] = 1
fitzdata.loc[fitzdata['diagnosis'].str.contains('basal cell carcinoma'), 'Other Diseases'] = 1
fitzdata.loc[fitzdata['diagnosis'].str.contains('melanosis'), 'Other Diseases'] = 1
fitzdata.loc[fitzdata['diagnosis'].str.contains('squamous cell carcinoma'), 'Other Diseases'] = 1
fitzdata.loc[fitzdata['diagnosis'].str.contains('actinic kera'), 'Other Diseases'] = 1
fitzdata.loc[fitzdata['diagnosis'].str.contains('neurofibromatosis'), 'Other Diseases'] = 1

fitzdata['Benign']='0'
fitzdata.loc[fitzdata['diagnosis'].str.contains('nevus'), 'Benign'] = 1
fitzdata.loc[fitzdata['diagnosis'].str.contains('seborrheic keratosis'), 'Benign'] = 1
fitzdata.loc[fitzdata['diagnosis'].str.contains('miscellaneous'), 'Benign'] = 1
fitzdata.loc[fitzdata['diagnosis'].str.contains('dermatofibroma'), 'Benign'] = 1
fitzdata.loc[fitzdata['diagnosis'].str.contains('nevi'), 'Benign'] = 1
fitzdata.loc[fitzdata['diagnosis'].str.contains('acne'), 'Benign'] = 1
fitzdata.loc[fitzdata['diagnosis'].str.contains('rosacea'), 'Benign'] = 1
fitzdata.loc[fitzdata['diagnosis'].str.contains('atopic'), 'Benign'] = 1 # atopic dermatitus
fitzdata.loc[fitzdata['diagnosis'].str.contains('Atopic'), 'Benign'] = 1 
fitzdata.loc[fitzdata['diagnosis'].str.contains('benign'), 'Benign'] = 1 
fitzdata.loc[fitzdata['diagnosis'].str.contains('dermatitis'), 'Benign'] = 1 
fitzdata.loc[fitzdata['diagnosis'].str.contains('eczema'), 'Benign'] = 1 
fitzdata.loc[fitzdata['diagnosis'].str.contains('dermatofibroma'), 'Benign'] = 1
fitzdata.loc[fitzdata['diagnosis'].str.contains('naevus'), 'Benign'] = 1

fitzdata['Other Diseases']=fitzdata['Other Diseases'].astype(int)
fitzdata.MEL=fitzdata.MEL.astype(int)
fitzdata.Benign=fitzdata.Benign.astype(int)

In [161]:
print(fitzdata.MEL.sum(),fitzdata.Benign.sum(),fitzdata['Other Diseases'].sum())

573 2457 1541


In [162]:
fitzdata=fitzdata[~(fitzdata[["MEL","Benign","Other Diseases"]] == 0).all(axis=1)]

In [163]:
fitzdata=fitzdata[["image","MEL","Benign","Other Diseases"]]


In [164]:
df=pd.concat([df,fitzdata])

In [155]:
src_path = "C:/Users/Migue/Desktop/thesis_data/finalfitz17k"
dst_path = 'C:/Users/Migue/Desktop/thesis_data/centralized_images'
x=0
for subdir, dirs, files in os.walk(src_path):
    for file in files:
        if file in fitzdata["image"].values:
            print(f"- {file}")
            src_file = os.path.join(src_path, file)
            dst_file = os.path.join(dst_path, file)
            print(f"Sending file {src_file} to {dst_file}")
            print()
            shutil.copyfile(src_file, dst_file)

            x+=1
print(x)

- 0009d89a58dac26bb3c113744b7b4196.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\0009d89a58dac26bb3c113744b7b4196.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\0009d89a58dac26bb3c113744b7b4196.jpg

- 000b6317b3be6d504e212a50e4d5667f.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\000b6317b3be6d504e212a50e4d5667f.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\000b6317b3be6d504e212a50e4d5667f.jpg

- 001d22ff2543f95d2d38c18da0446c84.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\001d22ff2543f95d2d38c18da0446c84.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\001d22ff2543f95d2d38c18da0446c84.jpg

- 002100fdd81baa28c893ddcde6892e3e.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\002100fdd81baa28c893ddcde6892e3e.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\002100fdd81baa28c893ddcde6892e3e.jpg

- 0041c7551e027def40cf81ab8fe14a36.jpg
Sending file C:/Users/Migue/Deskt

- 02ac06de46ddadc68923e8c1bc8854d1.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\02ac06de46ddadc68923e8c1bc8854d1.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\02ac06de46ddadc68923e8c1bc8854d1.jpg

- 02b1a2a60d2b3b6b2e5d15d1f48eab33.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\02b1a2a60d2b3b6b2e5d15d1f48eab33.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\02b1a2a60d2b3b6b2e5d15d1f48eab33.jpg

- 02f11821915d67f20fb15a8e8b96c1d3.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\02f11821915d67f20fb15a8e8b96c1d3.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\02f11821915d67f20fb15a8e8b96c1d3.jpg

- 02fd9e1b56e0c3adae18233f430f2fea.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\02fd9e1b56e0c3adae18233f430f2fea.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\02fd9e1b56e0c3adae18233f430f2fea.jpg

- 030ab080d0a380f3f5711d981b4dba98.jpg
Sending file C:/Users/Migue/Deskt

- 06d0d4dacbde9242cb23d5879414889d.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\06d0d4dacbde9242cb23d5879414889d.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\06d0d4dacbde9242cb23d5879414889d.jpg

- 06d1d14eb33eb1dd78409525b0829f51.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\06d1d14eb33eb1dd78409525b0829f51.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\06d1d14eb33eb1dd78409525b0829f51.jpg

- 06da798e65e2b0ae45a7ae3e14b344f0.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\06da798e65e2b0ae45a7ae3e14b344f0.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\06da798e65e2b0ae45a7ae3e14b344f0.jpg

- 06e752d8fe7b1970771d1dab779fa01c.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\06e752d8fe7b1970771d1dab779fa01c.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\06e752d8fe7b1970771d1dab779fa01c.jpg

- 070456f5185739adcf48e829cc6d562d.jpg
Sending file C:/Users/Migue/Deskt

- 08df04c5cea40bb194f1d303f97c301f.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\08df04c5cea40bb194f1d303f97c301f.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\08df04c5cea40bb194f1d303f97c301f.jpg

- 08e991228160baf31d10436c34108f77.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\08e991228160baf31d10436c34108f77.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\08e991228160baf31d10436c34108f77.jpg

- 08feca025dfb53bff8826d9c78c5fde8.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\08feca025dfb53bff8826d9c78c5fde8.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\08feca025dfb53bff8826d9c78c5fde8.jpg

- 093fcc49c98d314303ba07aa0d6c1348.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\093fcc49c98d314303ba07aa0d6c1348.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\093fcc49c98d314303ba07aa0d6c1348.jpg

- 095557969a0b98ecb06ab0051eadabcf.jpg
Sending file C:/Users/Migue/Deskt

- 0db67ec1711c7ab9d13c8fef53e527ea.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\0db67ec1711c7ab9d13c8fef53e527ea.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\0db67ec1711c7ab9d13c8fef53e527ea.jpg

- 0dd2cb24c519040fca7c77ed6609afed.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\0dd2cb24c519040fca7c77ed6609afed.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\0dd2cb24c519040fca7c77ed6609afed.jpg

- 0dd4fb7e120caac9f1f7db96eae103f6.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\0dd4fb7e120caac9f1f7db96eae103f6.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\0dd4fb7e120caac9f1f7db96eae103f6.jpg

- 0e040061170c554a1945be51928f7e3e.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\0e040061170c554a1945be51928f7e3e.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\0e040061170c554a1945be51928f7e3e.jpg

- 0e3b5b7c2a4852017252c865ef6b09b0.jpg
Sending file C:/Users/Migue/Deskt

...

- f1fa0c1a7632dfa036777befc290cc5f.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\f1fa0c1a7632dfa036777befc290cc5f.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\f1fa0c1a7632dfa036777befc290cc5f.jpg

- f21f7b5ec113e2fb4e705e550e918077.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\f21f7b5ec113e2fb4e705e550e918077.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\f21f7b5ec113e2fb4e705e550e918077.jpg

- f2327dc6bf154de3dec0dbdb99f4468e.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\f2327dc6bf154de3dec0dbdb99f4468e.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\f2327dc6bf154de3dec0dbdb99f4468e.jpg

- f23d762492106d7cfb3fdb1a3b4c37a6.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\f23d762492106d7cfb3fdb1a3b4c37a6.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\f23d762492106d7cfb3fdb1a3b4c37a6.jpg

- f2404024ad9adf7355a5363efe418567.jpg
Sending file C:/Users/Migue/Deskt

- f420eb73f0b50a89b5fddccc85796062.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\f420eb73f0b50a89b5fddccc85796062.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\f420eb73f0b50a89b5fddccc85796062.jpg

- f44c2e98d9f37b5f2c3a7e510ac0464b.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\f44c2e98d9f37b5f2c3a7e510ac0464b.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\f44c2e98d9f37b5f2c3a7e510ac0464b.jpg

- f44f3454329e674663e68da31c24e674.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\f44f3454329e674663e68da31c24e674.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\f44f3454329e674663e68da31c24e674.jpg

- f45dc98e818745e359d7d25b00c91151.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\f45dc98e818745e359d7d25b00c91151.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\f45dc98e818745e359d7d25b00c91151.jpg

- f4691ed2d1bddd01e929272db72e22c4.jpg
Sending file C:/Users/Migue/Deskt

- f861cde050f76eb47972ee60c43d8e90.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\f861cde050f76eb47972ee60c43d8e90.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\f861cde050f76eb47972ee60c43d8e90.jpg

- f86b01eff7cbd7a68926ba62beebd11b.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\f86b01eff7cbd7a68926ba62beebd11b.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\f86b01eff7cbd7a68926ba62beebd11b.jpg

- f8b65c26838a569b000c790e1d3542fa.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\f8b65c26838a569b000c790e1d3542fa.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\f8b65c26838a569b000c790e1d3542fa.jpg

- f8c6615c8c5af9d278d291b67f12c37e.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\f8c6615c8c5af9d278d291b67f12c37e.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\f8c6615c8c5af9d278d291b67f12c37e.jpg

- f8ce28c0e5cbf90fcbfe0ddde26bd027.jpg
Sending file C:/Users/Migue/Deskt

- fa866bb459eb719101f756a3b6244ca4.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\fa866bb459eb719101f756a3b6244ca4.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\fa866bb459eb719101f756a3b6244ca4.jpg

- fa874a1a22d0ac96cd101b8c492ce962.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\fa874a1a22d0ac96cd101b8c492ce962.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\fa874a1a22d0ac96cd101b8c492ce962.jpg

- fa89cc0aa31e87f03da6da8517264535.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\fa89cc0aa31e87f03da6da8517264535.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\fa89cc0aa31e87f03da6da8517264535.jpg

- fa9d0b0dbf1029f4bd1d3c07c536f417.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\fa9d0b0dbf1029f4bd1d3c07c536f417.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\fa9d0b0dbf1029f4bd1d3c07c536f417.jpg

- fa9ee8c99370dc53a1ddcf892410a2ea.jpg
Sending file C:/Users/Migue/Deskt

- fe950b66fca0c5edfcdcaa90c2cb4792.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\fe950b66fca0c5edfcdcaa90c2cb4792.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\fe950b66fca0c5edfcdcaa90c2cb4792.jpg

- fe95127aa67f455fb965e42e6c434478.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\fe95127aa67f455fb965e42e6c434478.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\fe95127aa67f455fb965e42e6c434478.jpg

- fea17cde96eafc8a203b008204083ace.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\fea17cde96eafc8a203b008204083ace.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\fea17cde96eafc8a203b008204083ace.jpg

- feaba9835c2ae53127581c14e59ab6ad.jpg
Sending file C:/Users/Migue/Desktop/thesis_data/finalfitz17k\feaba9835c2ae53127581c14e59ab6ad.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\feaba9835c2ae53127581c14e59ab6ad.jpg

- feabe7afd5bf64029cde195e8edad3a5.jpg
Sending file C:/Users/Migue/Deskt

# sd-260+sd198 dataset- hand organized folders


MEL- malignant and lentigo melanoma
OTHER Diseases- BCC scc metastatic carcinoma
Benign- nevi nevus acne benign keratosis dermatfibroma dermatosis

In [165]:
Benign = "C:/Users/Migue/Desktop/thesis_data/SD MERGE/Benign"
MEL = "C:/Users/Migue/Desktop/thesis_data/SD MERGE/Melanoma"
Other_Diseases = "C:/Users/Migue/Desktop/thesis_data/SD MERGE/Other Diseases"
dst_path = 'C:/Users/Migue/Desktop/thesis_data/centralized_images'

_Benign=[]
_MEL=[]
_Otherd=[]
for subdir, dirs, files in os.walk(Benign):
    for file in files:
        print(f"-Appending {file}... ")
        _Benign.append(file)
        
        src_file = os.path.join(Benign, file)
        dst_file = os.path.join(dst_path, file)
        print(f"Sending {file} {src_file} to {dst_file}")
        #shutil.copyfile(src_file, dst_file)   
        
for subdir, dirs, files in os.walk(MEL):
    for file in files:
        print(f"-Appending {file}... ")
        _MEL.append(file)
        src_file = os.path.join(MEL, file)
        dst_file = os.path.join(dst_path, file)
        print(f"Sending {file} {src_file} to {dst_file}")
        #shutil.copyfile(src_file, dst_file) 
              
for subdir, dirs, files in os.walk(Other_Diseases):
    for file in files:
        print(f"-Appending {file}... ")
        _Otherd.append(file)
        src_file = os.path.join(Other_Diseases, file)
        dst_file = os.path.join(dst_path, file)
        print(f"Sending {file} {src_file} to {dst_file}")
        #shutil.copyfile(src_file, dst_file)

-Appending 000214HB.jpg... 
Sending 000214HB.jpg C:/Users/Migue/Desktop/thesis_data/SD MERGE/Benign\000214HB.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\000214HB.jpg
-Appending 000215VB.jpg... 
Sending 000215VB.jpg C:/Users/Migue/Desktop/thesis_data/SD MERGE/Benign\000215VB.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\000215VB.jpg
-Appending 000216VB.jpg... 
Sending 000216VB.jpg C:/Users/Migue/Desktop/thesis_data/SD MERGE/Benign\000216VB.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\000216VB.jpg
-Appending 000220HB.jpg... 
Sending 000220HB.jpg C:/Users/Migue/Desktop/thesis_data/SD MERGE/Benign\000220HB.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\000220HB.jpg
-Appending 000221HB.jpg... 
Sending 000221HB.jpg C:/Users/Migue/Desktop/thesis_data/SD MERGE/Benign\000221HB.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\000221HB.jpg
-Appending 000222HB.jpg... 
Sending 000222HB.jpg C:/Users/Migue/Desktop/thesis_data/S

Sending 024884VB.jpg C:/Users/Migue/Desktop/thesis_data/SD MERGE/Benign\024884VB.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\024884VB.jpg
-Appending 024885HB.jpg... 
Sending 024885HB.jpg C:/Users/Migue/Desktop/thesis_data/SD MERGE/Benign\024885HB.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\024885HB.jpg
-Appending 024886VB.jpg... 
Sending 024886VB.jpg C:/Users/Migue/Desktop/thesis_data/SD MERGE/Benign\024886VB.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\024886VB.jpg
-Appending 024887VB.jpg... 
Sending 024887VB.jpg C:/Users/Migue/Desktop/thesis_data/SD MERGE/Benign\024887VB.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\024887VB.jpg
-Appending 024888VB.jpg... 
Sending 024888VB.jpg C:/Users/Migue/Desktop/thesis_data/SD MERGE/Benign\024888VB.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\024888VB.jpg
-Appending 024889HB.jpg... 
Sending 024889HB.jpg C:/Users/Migue/Desktop/thesis_data/SD MERGE/Benign\024889HB.jpg 

Sending 042797HB.jpg C:/Users/Migue/Desktop/thesis_data/SD MERGE/Benign\042797HB.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\042797HB.jpg
-Appending 042798VB.jpg... 
Sending 042798VB.jpg C:/Users/Migue/Desktop/thesis_data/SD MERGE/Benign\042798VB.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\042798VB.jpg
-Appending 042800HB.jpg... 
Sending 042800HB.jpg C:/Users/Migue/Desktop/thesis_data/SD MERGE/Benign\042800HB.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\042800HB.jpg
-Appending 042803VB.jpg... 
Sending 042803VB.jpg C:/Users/Migue/Desktop/thesis_data/SD MERGE/Benign\042803VB.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\042803VB.jpg
-Appending 042804VB.jpg... 
Sending 042804VB.jpg C:/Users/Migue/Desktop/thesis_data/SD MERGE/Benign\042804VB.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\042804VB.jpg
-Appending 042807HB.jpg... 
Sending 042807HB.jpg C:/Users/Migue/Desktop/thesis_data/SD MERGE/Benign\042807HB.jpg 

Sending 002579HB.jpg C:/Users/Migue/Desktop/thesis_data/SD MERGE/Other Diseases\002579HB.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\002579HB.jpg
-Appending 002580HB.jpg... 
Sending 002580HB.jpg C:/Users/Migue/Desktop/thesis_data/SD MERGE/Other Diseases\002580HB.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\002580HB.jpg
-Appending 002582HB.jpg... 
Sending 002582HB.jpg C:/Users/Migue/Desktop/thesis_data/SD MERGE/Other Diseases\002582HB.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\002582HB.jpg
-Appending 002583HB.jpg... 
Sending 002583HB.jpg C:/Users/Migue/Desktop/thesis_data/SD MERGE/Other Diseases\002583HB.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\002583HB.jpg
-Appending 002584HB.jpg... 
Sending 002584HB.jpg C:/Users/Migue/Desktop/thesis_data/SD MERGE/Other Diseases\002584HB.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\002584HB.jpg
-Appending 002585HB.jpg... 
Sending 002585HB.jpg C:/Users/Migue/Desktop/t

In [166]:
mel = pd.DataFrame({'image': _MEL,'MEL': 1})
benign = pd.DataFrame({'image': _Benign,'Benign': 1})
oth = pd.DataFrame({'image': _Otherd,'Other Diseases': 1})
SDmerge=pd.concat([mel,benign,oth])
SDmerge = SDmerge.fillna(0)
SDmerge.MEL=SDmerge.MEL.astype(int)
SDmerge.Benign=SDmerge.Benign.astype(int)
SDmerge["Other Diseases"]=SDmerge["Other Diseases"].astype(int)


In [167]:
print(SDmerge.MEL.sum(),SDmerge.Benign.sum(),SDmerge['Other Diseases'].sum())

373 5812 2503


In [168]:
df=pd.concat([df,SDmerge])


In [169]:
df

image MEL Benign Other Diseases
0     ISIC_0000000.jpg   0      1              0
1     ISIC_0000001.jpg   0      1              0
2     ISIC_0000002.jpg   1      0              0
3     ISIC_0000003.jpg   0      1              0
4     ISIC_0000004.jpg   1      0              0
...                ...  ..    ...            ...
2498      601234-2.jpg   0      0              1
2499      601234-3.jpg   0      0              1
2500      601234-4.jpg   0      0              1
2501      601234-5.jpg   0      0              1
2502      691131-1.jpg   0      0              1

[77976 rows x 4 columns]

# dermquest

In [170]:
Benign = "C:/Users/Migue/Desktop/thesis_data/dermquest/Not Melanoma"
MEL = "C:/Users/Migue/Desktop/thesis_data/dermquest/Melanoma"

In [171]:
_Benign=[]
_MEL=[]

for subdir, dirs, files in os.walk(Benign):
    for file in files:
        print(f"-Appending {file}... ")
        _Benign.append(file)
        
        src_file = os.path.join(Benign, file)
        dst_file = os.path.join(dst_path, file)
        print(f"Sending {file} {src_file} to {dst_file}")
        #shutil.copyfile(src_file, dst_file)   
        
for subdir, dirs, files in os.walk(MEL):
    for file in files:
        print(f"-Appending {file}... ")
        _MEL.append(file)
        src_file = os.path.join(MEL, file)
        dst_file = os.path.join(dst_path, file)
        print(f"Sending {file} {src_file} to {dst_file}")
        #shutil.copyfile(src_file, dst_file) 
mel = pd.DataFrame({'image': _MEL,'MEL': 1})
benign = pd.DataFrame({'image': _Benign,'Benign': 1})

dermquest=pd.concat([mel,benign])
dermquest = dermquest.fillna(0)
dermquest.MEL=dermquest.MEL.astype(int)
dermquest.Benign=dermquest.Benign.astype(int)
dermquest["Other Diseases"]=0


-Appending D10_1_orig.jpg... 
Sending D10_1_orig.jpg C:/Users/Migue/Desktop/thesis_data/dermquest/Not Melanoma\D10_1_orig.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\D10_1_orig.jpg
-Appending D10_orig.jpg... 
Sending D10_orig.jpg C:/Users/Migue/Desktop/thesis_data/dermquest/Not Melanoma\D10_orig.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\D10_orig.jpg
-Appending D11_2_orig.jpg... 
Sending D11_2_orig.jpg C:/Users/Migue/Desktop/thesis_data/dermquest/Not Melanoma\D11_2_orig.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\D11_2_orig.jpg
-Appending D11_orig.jpg... 
Sending D11_orig.jpg C:/Users/Migue/Desktop/thesis_data/dermquest/Not Melanoma\D11_orig.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\D11_orig.jpg
-Appending D15_orig.jpg... 
Sending D15_orig.jpg C:/Users/Migue/Desktop/thesis_data/dermquest/Not Melanoma\D15_orig.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\D15_orig.jpg
-Appending D16_2_orig.jpg... 
Send

In [172]:
df=pd.concat([df,dermquest])

In [173]:
df.drop_duplicates(inplace=True)
df=df.fillna(0)
df.MEL=df.MEL.astype(int)
df.Benign=df.Benign.astype(int)
df["Other Diseases"]=df["Other Diseases"].astype(int)

In [194]:

# get a list of all the filenames in a specified directory
directory = "C:/Users/Migue/Desktop/thesis_data/centralized_images"
filenames = os.listdir(directory)

# check if each filename in the folder is also in the "image" column of your DataFrame
for filename in filenames:
    if filename not in df["image"].values:
        print(f"Filename '{filename}' not found in DataFrame!")

In [174]:
df["image"] = df["image"].apply(lambda x: x + ".bmp" if "IMD" in x else x)

# med-node

In [ ]:
import shutil

In [175]:
Benign = "C:/Users/Migue/Desktop/thesis_data/complete_mednode_dataset/Benign"
MEL = "C:/Users/Migue/Desktop/thesis_data/complete_mednode_dataset/melanoma"
dst_path = 'C:/Users/Migue/Desktop/thesis_data/centralized_images'

In [176]:
_Benign=[]
_MEL=[]

for subdir, dirs, files in os.walk(Benign):
    for file in files:
        print(f"-Appending {file}... ")
        _Benign.append(file)
        
        src_file = os.path.join(Benign, file)
        dst_file = os.path.join(dst_path, file)
        print(f"Sending {file} {src_file} to {dst_file}")
        #shutil.copyfile(src_file, dst_file)   
        
for subdir, dirs, files in os.walk(MEL):
    for file in files:
        print(f"-Appending {file}... ")
        _MEL.append(file)
        src_file = os.path.join(MEL, file)
        dst_file = os.path.join(dst_path, file)
        print(f"Sending {file} {src_file} to {dst_file}")
        #shutil.copyfile(src_file, dst_file) 
mel = pd.DataFrame({'image': _MEL,'MEL': 1})
benign = pd.DataFrame({'image': _Benign,'Benign': 1})

mednode=pd.concat([mel,benign])

-Appending 132357.jpg... 
Sending 132357.jpg C:/Users/Migue/Desktop/thesis_data/complete_mednode_dataset/Benign\132357.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\132357.jpg
-Appending 151560-h.jpg... 
Sending 151560-h.jpg C:/Users/Migue/Desktop/thesis_data/complete_mednode_dataset/Benign\151560-h.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\151560-h.jpg
-Appending 156136.jpg... 
Sending 156136.jpg C:/Users/Migue/Desktop/thesis_data/complete_mednode_dataset/Benign\156136.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\156136.jpg
-Appending 159033.jpg... 
Sending 159033.jpg C:/Users/Migue/Desktop/thesis_data/complete_mednode_dataset/Benign\159033.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\159033.jpg
-Appending 162029.jpg... 
Sending 162029.jpg C:/Users/Migue/Desktop/thesis_data/complete_mednode_dataset/Benign\162029.jpg to C:/Users/Migue/Desktop/thesis_data/centralized_images\162029.jpg
-Appending 163067.jpg... 
Sending 163

In [177]:
df=pd.concat([df,mednode])

In [178]:
df.drop_duplicates(inplace=True)
df=df.fillna(0)
df.MEL=df.MEL.astype(int)
df.Benign=df.Benign.astype(int)
df["Other Diseases"]=df["Other Diseases"].astype(int)

In [179]:
df[df[["MEL","Benign", "Other Diseases"]].eq(0).all(axis=1)]

Empty DataFrame
Columns: [image, MEL, Benign, Other Diseases]
Index: []

In [181]:
print(df.MEL.sum(),df.Benign.sum(),df['Other Diseases'].sum())

7226 60788 10269


In [190]:
df.to_csv('centralized_truth.csv', index=False)